In [1]:
!pip3 install torch

In [2]:
!pip3 install sentencepiece

In [3]:
!pip3 install transformers

In [4]:
# Importing libraries
import json
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

In [5]:
!nvidia-smi

Wed Dec 13 17:12:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
get_ipython().system('nvidia-smi')

Wed Dec 13 17:12:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda:0' if cuda.is_available() else 'cpu'

In [8]:
print(device)

cuda:0


In [9]:
class YourDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

In [10]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 500 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [11]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    inputs = []
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
            

            generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=512, 
                min_length = 250,
              num_beams = 5,
              no_repeat_ngram_size = 5,
              #topp = 0.9,
              #do_sample=True,
              repetition_penalty=5.8, 
              length_penalty=1, 
              early_stopping=True
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            input_text = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in ids]
            if _%50==0:
                outputs = model(
                        input_ids=ids,
                        attention_mask=mask,
                        decoder_input_ids=y_ids,
                        labels=lm_labels,
                        )
                loss = outputs[0]
                console.print(f'Completed {_}')
                console.print('loss: '+ str(loss))
            
            predictions.extend(preds)
            actuals.extend(target)
            inputs.extend(input_text)
    return inputs, predictions, actuals


In [12]:
def generate(tokenizer, model, device, loader):
    model.eval()
    inputs = []
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
            

            generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=512, 
                min_length = 100,
              num_beams = 4,
              no_repeat_ngram_size = 5,
              #topp = 0.9,
              #do_sample=True,
              repetition_penalty=5.8, 
              length_penalty=1, 
              early_stopping=True
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            input_text = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in ids]
            if _%50==0:
                console.print(f'Completed {_}')

            predictions.extend(preds)
            inputs.extend(input_text)
    return inputs, predictions

In [13]:
def BartTrainer(
    dataframe, source_text, target_text, model_params, model, tokenizer, output_dir="./outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So x% of the data will be used for training and the rest for validation.
    train_size = 0.998
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = YourDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = YourDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)
        console.log(f"[Initiating Validation]...\n")
        inputs, predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Input': inputs, "Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions"+str(epoch)+".csv"))

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [14]:
# files=['news_yake_keywords.json','roc_yake_keywords.json', 'poetry_yake_keyword.json']

# def merge_JsonFiles(filename):
#     result = list()
#     for f1 in filename:
#         with open(f1, 'r') as infile:
#             result.append(json.load(infile))

#     with open('merged_yake_keywords.json', 'w') as output_file:
#         json.dump(result, output_file)

# merge_JsonFiles(files)



In [15]:
# f = open('news_yake_keywords.json', errors='ignore').readlines()
# news =  json.loads(f[0])
# g = open('roc_yake_keywords.json', errors='ignore').readlines()
# roc =  json.loads(g[0])
# h = open('poetry_yake_keywords.json', errors='ignore').readlines()
# poetry =  json.loads(g[0])
# all_scary =  json.loads(f[0])

In [16]:
# len(news), len(roc), len(poetry)

In [17]:
# all_poetry_foundation = roc + poetry + news

In [18]:
df = pd.read_csv('shakespeare.csv')

In [19]:
 # Replace with the desired number of columns
df = df.iloc[:20000]

In [20]:
len(df)

20000

In [21]:
df.head()

,Unnamed: 0,id,og,t
0,0,42928-1500614319216-63344,You do not meet a man but frowns:,Every man you meet these days is frowning.
1,1,42928-1500614326583-89821,our bloods No more obey the heavens than our...,Our bodies are in agreement with the planetar...
2,2,A-63849,But what's the matter?,What's wrong?
3,3,42930-1500614347266-80123,"His daughter, and the heir of's kingdom, whom...","The king wanted his daughter, the only heir to..."
4,4,42930-1500614355280-38326,she's wedded; Her husband banish'd; she impr...,"She's married, her husband is banished, she's..."


In [22]:

df = df.rename(columns={'t': 'title'})
df = df.rename(columns={'og': 'keywords'})

#Create a new DataFrame with only two columns
df = df[['title', 'keywords']]
df.head()

,title,keywords
0,Every man you meet these days is frowning.,You do not meet a man but frowns:
1,Our bodies are in agreement with the planetar...,our bloods No more obey the heavens than our...
2,What's wrong?,But what's the matter?
3,"The king wanted his daughter, the only heir to...","His daughter, and the heir of's kingdom, whom..."
4,"She's married, her husband is banished, she's...",she's wedded; Her husband banish'd; she impr...


In [23]:
import string

def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    
    # Remove symbols and numbers
    text = ''.join([char for char in text if char.isalpha() or char.isspace()])
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text


# Apply the clean_text function to the 'Text' column
df['title'] = df['title'].apply(clean_text)
# Apply the clean_text function to the 'Text' column
df['keywords'] = df['keywords'].apply(clean_text)

In [24]:
# Text to prepend
text_to_prepend = 'Translate English to Shakespearean: '

# Prepend text to the beginning of each row in the specified column
df['title'] = text_to_prepend + df['title']

In [25]:
df.head()

,title,keywords
0,Translate English to Shakespearean: every man ...,you do not meet a man but frowns
1,Translate English to Shakespearean: our bodies...,our bloods no more obey the heavens than our c...
2,Translate English to Shakespearean: whats wrong,but whats the matter
3,Translate English to Shakespearean: the king w...,his daughter and the heir ofs kingdom whom he ...
4,Translate English to Shakespearean: shes marri...,shes wedded her husband banishd she imprisond ...


In [26]:
# special_token = '.'
# mask_token = '<MASK>'
# eos_token = '</s>'

# X_titles = []
# y_keywords = []
# template = [mask_token, mask_token, mask_token]
# prompt = 'Generate keywords for the title: '
# title_set = []

# for poem in all_poetry_foundation:
#     title_set.append(poem['Theme'])
#     title = prompt + poem['Theme']
#     paddings = []
#     temp = []
#     count = 0
#     for key in poem['keywords']:
#         if key == ['<paragraph>']:
#             continue
#         count += 1
#         mask = template[:len(key)]
#         paddings.append('Keywords '+ str(count) + ': '+ str(mask) )        
#         temp.append('Keywords '+ str(count) + ': '+ str(key))

#     paddings = (" "+special_token+" ").join(paddings).replace('<paragraph> ','')
#     temp = (" "+special_token+" ").join(temp).replace('<paragraph> ','')

#     X_titles.append(title + '. ' + paddings)
#     #X_titles.append(title)
#     y_keywords.append(temp) 


In [27]:
# data = [X_titles, y_keywords]
# df = pd.DataFrame(np.array(data).T, columns = ['title', 'keywords'])
df['title'].head()

0    Translate English to Shakespearean: every man ...
1    Translate English to Shakespearean: our bodies...
2      Translate English to Shakespearean: whats wrong
3    Translate English to Shakespearean: the king w...
4    Translate English to Shakespearean: shes marri...
Name: title, dtype: object

In [28]:
df['title'][10005]

'Translate English to Shakespearean: im a virgin sir and though ive never asked for anyones attention ive had a lot of people stare at me as though i were a shooting star'

In [29]:
df['keywords'][10005]

'i am a maid my lord that neer before invited eyes but have been gazed on like a comet'

In [30]:
# let's define model parameters specific to bart
model_params = {
    "TASK" : "seq2seq",
    "MODEL": "facebook/bart-large",  
    "TRAIN_BATCH_SIZE": 2,  # training batch size
    "VALID_BATCH_SIZE": 2,  # validation batch size
    "TRAIN_EPOCHS": 4,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 5e-6,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 512,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

In [31]:
# tokenzier for encoding the text
tokenizer = BartTokenizer.from_pretrained(model_params["MODEL"])

In [32]:
tokenizer.eos_token

'</s>'

In [33]:
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = BartForConditionalGeneration.from_pretrained(model_params["MODEL"])
model = model.to(device)

In [34]:
tokens = tokenizer(df['title'][15005])

len(tokens['input_ids'])

30

In [35]:
df['keywords'][15005]

'what mean you caesar think you to walk forthyou shall not stir out of your house today'

In [36]:
tokens = tokenizer(df['keywords'][15005])

len(tokens['input_ids'])

21

In [37]:
output_dir= model_params['MODEL']+"_batch_size_"+ str(model_params['TRAIN_BATCH_SIZE']) + "_lr_"+ str(model_params['LEARNING_RATE'])+ "_" + model_params['TASK']
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
BartTrainer(
    dataframe=df,
    source_text="title",
    target_text="keywords",
    model_params=model_params,
    model = model,
    tokenizer = tokenizer,
    output_dir = output_dir
)

[17:12:44] [Model]: Loading facebook/bart-large...                                                 ]8;id=261683;file:///var/tmp/ipykernel_294843/2315885737.py\2315885737.py]8;;\:]8;id=28417;file:///var/tmp/ipykernel_294843/2315885737.py#16\16]8;;\
                                                                                                                   

           [Data]: Reading data...                                                                 ]8;id=163173;file:///var/tmp/ipykernel_294843/2315885737.py\2315885737.py]8;;\:]8;id=578688;file:///var/tmp/ipykernel_294843/2315885737.py#21\21]8;;\
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|Translate English to Shakespearean: every man you meet  |            you do not meet a man but frowns            |
|                these days is frowning                  |                                                        |
| Translate English to Shakespearean: our bodies are in  | our bloods no more obey the heavens than our courtiers |
|  agreement with the planetary influences just as the   |               still seem as does the king              |
|      courtierss moods seem to reflect the kings        |                                                        |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (20000, 2)

TRAIN Dataset: (19960, 2)

...

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(9.9361, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  500  | tensor(3.7519, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1000  | tensor(1.4152, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1500  | tensor(3.2544, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2000  | tensor(4.8772, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2500  | tensor(3.4691, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3000  | tensor(3.1583, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3500  | tensor(2.8779, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4000  | tensor(3.4171, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4500  | tensor(3.0024, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5000  | tensor(3.5004, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5500  | tensor(4.0300, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6000  | tensor(2.0338, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6500  | tensor(3.5547, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7000  | tensor(3.4398, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7500  | tensor(3.9838, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8000  | tensor(3.0973, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8500  | tensor(1.8334, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9000  | tensor(2.4744, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9500  | tensor(3.2739, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(4.3466, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |  500  | tensor(1.8999, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1000  | tensor(1.4629, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1500  | tensor(2.2721, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2000  | tensor(2.5485, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2500  | tensor(2.0093, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3000  | tensor(1.2859, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3500  | tensor(4.0834, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4000  | tensor(2.1942, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4500  | tensor(3.3372, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5000  | tensor(4.4174, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5500  | tensor(3.6426, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6000  | tensor(3.1389, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6500  | tensor(4.5780, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 7000  | tensor(1.3882, device='cuda:0', grad_fn=<NllLossBackward0>)|
+---------------------------------------------------------------------------+

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(9.9361, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  500  | tensor(3.7519, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1000  | tensor(1.4152, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1500  | tensor(3.2544, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2000  | tensor(4.8772, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2500  | tensor(3.4691, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3000  | tensor(3.1583, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3500  | tensor(2.8779, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4000  | tensor(3.4171, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4500  | tensor(3.0024, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5000  | tensor(3.5004, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5500  | tensor(4.0300, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6000  | tensor(2.0338, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6500  | tensor(3.5547, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7000  | tensor(3.4398, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7500  | tensor(3.9838, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8000  | tensor(3.0973, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8500  | tensor(1.8334, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9000  | tensor(2.4744, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9500  | tensor(3.2739, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(4.3466, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |  500  | tensor(1.8999, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1000  | tensor(1.4629, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1500  | tensor(2.2721, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2000  | tensor(2.5485, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2500  | tensor(2.0093, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3000  | tensor(1.2859, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3500  | tensor(4.0834, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4000  | tensor(2.1942, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4500  | tensor(3.3372, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5000  | tensor(4.4174, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5500  | tensor(3.6426, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6000  | tensor(3.1389, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6500  | tensor(4.5780, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 7000  | tensor(1.3882, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 7500  | tensor(3.1831, device='cuda:0', grad_fn=<NllLossBackward0>)|
+---------------------------------------------------------------------------+

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(9.9361, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  500  | tensor(3.7519, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1000  | tensor(1.4152, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1500  | tensor(3.2544, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2000  | tensor(4.8772, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2500  | tensor(3.4691, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3000  | tensor(3.1583, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3500  | tensor(2.8779, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4000  | tensor(3.4171, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4500  | tensor(3.0024, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5000  | tensor(3.5004, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5500  | tensor(4.0300, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6000  | tensor(2.0338, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6500  | tensor(3.5547, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7000  | tensor(3.4398, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7500  | tensor(3.9838, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8000  | tensor(3.0973, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8500  | tensor(1.8334, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9000  | tensor(2.4744, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9500  | tensor(3.2739, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(4.3466, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |  500  | tensor(1.8999, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1000  | tensor(1.4629, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1500  | tensor(2.2721, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2000  | tensor(2.5485, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2500  | tensor(2.0093, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3000  | tensor(1.2859, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3500  | tensor(4.0834, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4000  | tensor(2.1942, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4500  | tensor(3.3372, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5000  | tensor(4.4174, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5500  | tensor(3.6426, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6000  | tensor(3.1389, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6500  | tensor(4.5780, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 7000  | tensor(1.3882, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 7500  | tensor(3.1831, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 8000  | tensor(2.6786, device='cuda:0', grad_fn=<NllLossBackward0>)|
+---------------------------------------------------------------------------+

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(9.9361, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  500  | tensor(3.7519, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1000  | tensor(1.4152, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1500  | tensor(3.2544, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2000  | tensor(4.8772, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2500  | tensor(3.4691, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3000  | tensor(3.1583, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3500  | tensor(2.8779, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4000  | tensor(3.4171, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4500  | tensor(3.0024, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5000  | tensor(3.5004, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5500  | tensor(4.0300, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6000  | tensor(2.0338, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6500  | tensor(3.5547, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7000  | tensor(3.4398, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7500  | tensor(3.9838, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8000  | tensor(3.0973, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8500  | tensor(1.8334, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9000  | tensor(2.4744, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9500  | tensor(3.2739, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(4.3466, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |  500  | tensor(1.8999, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1000  | tensor(1.4629, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1500  | tensor(2.2721, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2000  | tensor(2.5485, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2500  | tensor(2.0093, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3000  | tensor(1.2859, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3500  | tensor(4.0834, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4000  | tensor(2.1942, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4500  | tensor(3.3372, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5000  | tensor(4.4174, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5500  | tensor(3.6426, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6000  | tensor(3.1389, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6500  | tensor(4.5780, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 7000  | tensor(1.3882, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 7500  | tensor(3.1831, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 8000  | tensor(2.6786, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 8500  | tensor(1.7214, device='cuda:0', grad_fn=<NllLossBackward0>)|
+---------------------------------------------------------------------------+

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(9.9361, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  500  | tensor(3.7519, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1000  | tensor(1.4152, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 1500  | tensor(3.2544, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2000  | tensor(4.8772, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 2500  | tensor(3.4691, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3000  | tensor(3.1583, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 3500  | tensor(2.8779, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4000  | tensor(3.4171, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 4500  | tensor(3.0024, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5000  | tensor(3.5004, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 5500  | tensor(4.0300, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6000  | tensor(2.0338, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 6500  | tensor(3.5547, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7000  | tensor(3.4398, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 7500  | tensor(3.9838, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8000  | tensor(3.0973, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 8500  | tensor(1.8334, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9000  | tensor(2.4744, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   | 9500  | tensor(3.2739, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(4.3466, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   |  500  | tensor(1.8999, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1000  | tensor(1.4629, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 1500  | tensor(2.2721, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2000  | tensor(2.5485, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 2500  | tensor(2.0093, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3000  | tensor(1.2859, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 3500  | tensor(4.0834, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4000  | tensor(2.1942, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 4500  | tensor(3.3372, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5000  | tensor(4.4174, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 5500  | tensor(3.6426, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6000  | tensor(3.1389, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 6500  | tensor(4.5780, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 7000  | tensor(1.3882, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 7500  | tensor(3.1831, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 8000  | tensor(2.6786, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 8500  | tensor(1.7214, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  1   | 9000  | tensor(3.6410, device='cuda:0', grad_fn=<NllLossBackward0>)|
+---------------------------------------------------------------------------+